# Convert CSV to JSON

In [1]:
import pandas as pd
import json

def csv_to_json_dynamic(csv_file_path, output_json_path):
    # Load the CSV file
    df = pd.read_csv(csv_file_path)
    
    # Initialize the JSON data dictionary
    json_data = {}
    
    # Iterate through each row in the DataFrame
    for index, row in df.iterrows():
        entry = {}
        for column in df.columns:
            # Using column names directly from CSV
            entry[column] = row[column]
        
        # Use the first column as the key for JSON entries
        key = str(row[df.columns[0]])
        json_data[key] = entry
    
    # Save the converted data to a JSON file
    with open(output_json_path, 'w') as json_file:
        json.dump(json_data, json_file, indent=4)
    
    return output_json_path

# Path to the input CSV file
csv_file_path = 'data_csv/Data Capstone - Raw.csv'
# Path to the output JSON file
output_json_path = 'data_json/Capstone_RAG3.json'

# Convert CSV to JSON dynamically
converted_json_path = csv_to_json_dynamic(csv_file_path, output_json_path)
converted_json_path


'data_json/Capstone_RAG3.json'

# Load JSON data

In [2]:
import json

# Define the file path to the JSON file
file_path = r'data_json/Capstone_RAG3.json'

# Open the JSON file for reading with UTF-8 encoding
with open(file_path, 'r', encoding='utf-8') as file:
    # Load the JSON data from the file into a Python dictionary
    data = json.load(file)
    
    # Convert the values of the dictionary to a list
    eskwe_data = list(data.values())

    # Print the loaded JSON data in a nicely formatted way
    print("Loaded JSON data:")
    print(json.dumps(data, indent=4))


Loaded JSON data:
{
    "1": {
        "No.": 1,
        "Title": "Overview and Topics of Sprint 1",
        "Link": "https://docs.google.com/spreadsheets/d/1TuJEyzLfSoFf7ukmVg1pNSWApJHQNNhL_NtvxwN_nF8/edit?gid=1163492588#gid=1163492588",
        "Body": "Main Topic: Introduction to Data Science and Machine Learning\nSubtopics: \n-Python Fundamentals\n-Pandas: Data Wrangling Techniques\n-Data Distribtutions\n-Data Visualizations\n-Exploratory Data Analysis\n-Data Story Telling\n-Github\n-Deployment using Streamlit Cloud\n-Introduction to Machine Learning\n-RFM Clustering\n-Introduction to Linear Regression and Logistic Regression",
        "Author": "Eskwelabs",
        "Date Published": "Aug 1, 2024",
        "Sprint": "Sprint Topics",
        "Notes": "All sprints"
    },
    "2": {
        "No.": 2,
        "Title": "Overview and Topics of Sprint 2",
        "Link": "https://docs.google.com/spreadsheets/d/1TuJEyzLfSoFf7ukmVg1pNSWApJHQNNhL_NtvxwN_nF8/edit?gid=1163492588#gid=116349258

In [3]:
# Show the list of values
eskwe_data

[{'No.': 1,
  'Title': 'Overview and Topics of Sprint 1',
  'Link': 'https://docs.google.com/spreadsheets/d/1TuJEyzLfSoFf7ukmVg1pNSWApJHQNNhL_NtvxwN_nF8/edit?gid=1163492588#gid=1163492588',
  'Body': 'Main Topic: Introduction to Data Science and Machine Learning\nSubtopics: \n-Python Fundamentals\n-Pandas: Data Wrangling Techniques\n-Data Distribtutions\n-Data Visualizations\n-Exploratory Data Analysis\n-Data Story Telling\n-Github\n-Deployment using Streamlit Cloud\n-Introduction to Machine Learning\n-RFM Clustering\n-Introduction to Linear Regression and Logistic Regression',
  'Author': 'Eskwelabs',
  'Date Published': 'Aug 1, 2024',
  'Sprint': 'Sprint Topics',
  'Notes': 'All sprints'},
 {'No.': 2,
  'Title': 'Overview and Topics of Sprint 2',
  'Link': 'https://docs.google.com/spreadsheets/d/1TuJEyzLfSoFf7ukmVg1pNSWApJHQNNhL_NtvxwN_nF8/edit?gid=1163492588#gid=1163492588',
  'Body': 'Main topic: Machine Learning Techniques and Model Evaluation\nSubtopics:\n-Introduction to Credit 

# Vector Embeddings & Storage

In [2]:
# Import necessary libraries
from dotenv import load_dotenv  # For loading environment variables from a .env file
import os  # For interacting with the operating system
import openai  # For interacting with OpenAI's API
import json  # For working with JSON data
import chromadb  # For using ChromaDB
from chromadb.utils import embedding_functions  # Utility functions for embeddings in ChromaDB

# Load environment variables from .env file
load_dotenv()

# Set the OpenAI API key from environment variable
openai.api_key = os.getenv("OPENAI_API_KEY")

# Load eskwe_data JSON file
file_path = r'data_json/Capstone_RAG3.json'
with open(file_path, 'r', encoding='utf-8') as file:
    data = json.load(file)  # Load the JSON data from the file into a dictionary
    eskwe_data = list(data.values())  # Convert the values of the dictionary to a list

# Constants and Initializing ChromaDB
CHROMA_DATA_PATH = 'eskwe'  # Path for storing ChromaDB data
COLLECTION_NAME = "eskwe_embeddings"  # Name of the ChromaDB collection

# Initialize ChromaDB client
client = chromadb.PersistentClient(path=CHROMA_DATA_PATH)  # Create a persistent client for ChromaDB
openai_ef = embedding_functions.OpenAIEmbeddingFunction(api_key=os.getenv("OPENAI_API_KEY"), model_name="text-embedding-ada-002")  # Define the OpenAI embedding function

# Create or get the collection in ChromaDB
collection = client.get_or_create_collection(
    name=COLLECTION_NAME,
    embedding_function=openai_ef,
    metadata={"hnsw:space": "cosine"}  # Using cosine space for HNSW (Hierarchical Navigable Small World)
)

# Function to generate embeddings and add to collection
def add_embeddings_to_collection(eskwelabs):
    for i, eskdata in enumerate(eskwelabs):
        # Prepare the data for embedding
        eskwe_data_final = f"Title: {eskdata.get('Title', 'N/A')}\nLink: {eskdata.get('Link', 'N/A')}\nPublish Date: {eskdata.get('Date Published', 'N/A')}\nSprint: {eskdata.get('Sprint', 'N/A')}\nBody: {eskdata.get('Body', 'N/A')}"
        
        # Use OpenAI API to create embeddings
        response = openai.Embedding.create(
            input=eskwe_data_final, 
            model="text-embedding-ada-002"
        )
        embedding = response.data[0].embedding  # Extract the embedding from the response
        print(f"Generated embedding for eskdata {eskdata.get('Title', 'Unknown Data')}: {embedding[:5]}...")

        try:
            # Add the embedding to the ChromaDB collection
            collection.add(
                ids=[str(i)],
                documents=[eskwe_data_final],
                embeddings=[embedding],
                metadatas=[{"eskdata": eskdata.get('Title', 'Unknown Data')}]
            )
            print(f"Stored embedding for eskdata {eskdata.get('Title', 'Unknown Data')}")
        except Exception as e:  # Catch potential errors in collection storage
            print(f"Error storing embedding for eskdata {eskdata.get('Title', 'Unknown Data')}: {e}")

# Function to retrieve and print embedding and metadata by ID
def retrieve_embedding_by_id(doc_id):
    try:
        query_result = collection.get(ids=[str(doc_id)], include=['embeddings', 'metadatas', 'documents'])
        print(f"Query result for ID {doc_id}: {query_result}")

        if not query_result or 'ids' not in query_result or not query_result['ids']:
            print(f"No results found for the given ID: {doc_id}.")
            return None, None, None  # Return None for all if not found

        embedding = query_result['embeddings'][0] if query_result['embeddings'] else None
        metadata = query_result['metadatas'][0] if query_result['metadatas'] else None
        document = query_result['documents'][0] if query_result['documents'] else None
        return embedding, metadata, document

    except Exception as e:
        print(f"Error retrieving embedding for ID {doc_id}: {str(e)}")
        return None, None, None  # Return None for all in case of error

# Add embeddings to the collection
add_embeddings_to_collection(eskwe_data)

# Verify retrieval
print("\nStored Embeddings in ChromaDB:")
for i in range(min(10, len(eskwe_data))):  # Iterate up to 10 or the number of data, whichever is smaller
    embedding, metadata, document = retrieve_embedding_by_id(i)
    if embedding and metadata:
        print(f"ID: {i}, Embedding (first 5): {embedding[:5]}..., Metadata: {metadata}, Document(first 100 chars): {document[:100]}")
    else:
        print(f"Failed to retrieve embedding and metadata for ID: {i}")


Add of existing embedding ID: 0
Insert of existing embedding ID: 0


Generated embedding for eskdata Overview and Topics of Sprint 1: [-0.003084039082750678, -0.001510584494099021, 0.016013214364647865, 0.006871545221656561, -0.016951169818639755]...
Stored embedding for eskdata Overview and Topics of Sprint 1


Add of existing embedding ID: 1
Insert of existing embedding ID: 1


Generated embedding for eskdata Overview and Topics of Sprint 2: [-0.014404241926968098, 0.013667123392224312, 0.022343480959534645, 0.007276508957147598, -0.009088873863220215]...
Stored embedding for eskdata Overview and Topics of Sprint 2


Add of existing embedding ID: 2
Insert of existing embedding ID: 2


...

Generated embedding for eskdata Unsupervised Learning and Data Clustering: [0.002182227559387684, 0.025530723854899406, 0.013394592329859734, -0.017136510461568832, 0.009860187768936157]...
Stored embedding for eskdata Unsupervised Learning and Data Clustering


Add of existing embedding ID: 58
Insert of existing embedding ID: 58


Generated embedding for eskdata Top 50 Matplotlib Visualizations – The Master Plots (with full Python code)
: [-0.0027891010977327824, -0.004495021887123585, 0.0073035284876823425, 0.0018205896485596895, -0.004135137889534235]...
Stored embedding for eskdata Top 50 Matplotlib Visualizations – The Master Plots (with full Python code)



Add of existing embedding ID: 59
Insert of existing embedding ID: 59


Generated embedding for eskdata Introduction to Data Preprocessing in Machine Learning: [-0.004934465046972036, 0.027075255289673805, 0.019480645656585693, -0.009956925176084042, -0.012765982188284397]...
Stored embedding for eskdata Introduction to Data Preprocessing in Machine Learning

Stored Embeddings in ChromaDB:
Query result for ID 0: {'ids': ['0'], 'embeddings': [[-0.003084039082750678, -0.001510584494099021, 0.016013214364647865, 0.006871545221656561, -0.016951169818639755, -0.010439855046570301, 0.0036226839292794466, 0.0018351307371631265, -0.03610449656844139, -0.038061968982219696, 0.024454815313220024, 0.020104875788092613, -0.026167605072259903, 0.014816981740295887, 0.0005424680421128869, 0.004108653869479895, 0.015197601169347763, -0.03208080306649208, -0.006045736372470856, -0.023571234196424484, -0.015782123431563377, -0.008428007364273071, -0.0015352227492257953, 0.0022072545252740383, -0.011343826539814472, 0.010738912969827652, 0.014246052131056786, -0.02005050145

In [3]:
# Retrieve Embedding by ID
# Verify a specific ID (5 in this case)
'''This line calls the retrieve_embedding_by_id function with the argument 5, which is the ID of the data we want to verify. 
The function returns three values: embedding, metadata, and document.'''
embedding, metadata, document = retrieve_embedding_by_id(5)  # Unpack all three values

# Check if Data Exists:
'''This line checks if all three values (embedding, metadata, and document) are not None. 
If any of these values is None, it means the data for the specified ID was not found or an error occurred.'''
if embedding and metadata and document:
    # Print Data if Exists:
    '''If the data exists, this line prints the ID, the first five elements of the embedding, the metadata, and the first 100 characters of the document.'''
    print(f"ID: 5, Embedding: {embedding[:5]}..., Metadata: {metadata}, Document (first 100 chars): {document[:100]}...")  # Include document

# Print Error Message if Data Not Found:
# If the data does not exist, this line prints an error message indicating the failure to retrieve data for the specified ID.
else:
    print(f"Failed to retrieve embedding and metadata for ID: 5")

Query result for ID 5: {'ids': ['5'], 'embeddings': [[-0.006038852035999298, -0.006417734548449516, 0.016883527860045433, -0.008568057790398598, -0.00540738133713603, -0.015155292116105556, -0.012330292724072933, -0.022493643686175346, -0.004892234690487385, -0.02037987858057022, 0.014570350758731365, 0.0006792462081648409, -0.029619289562106133, 0.013187762349843979, -0.013599880039691925, 0.0015670438297092915, 0.016125762835144997, -0.005852734204381704, -0.01389235071837902, -0.00820246897637844, -0.025298701599240303, 0.01068846881389618, 0.014689997769892216, -0.013407115824520588, -0.016338467597961426, 0.008435116149485111, 0.009066587314009666, -0.03778187558054924, 0.011373115703463554, -0.014105056412518024, 0.019422702491283417, -0.015873173251748085, -0.015407879836857319, -0.029645878821611404, -0.01018993929028511, 0.007757116574794054, 0.0076308222487568855, 0.003964969888329506, 0.0319058783352375, 0.022932348772883415, 0.025976702570915222, 0.005540322512388229, -0.01

# Query Vector Storage

In [3]:
# Query the collection for texts similar to "RAG" and get the top 5 results
# This line queries the ChromaDB collection to find the top 5 results that are most similar to the query text "RAG".
query_result = collection.query(query_texts=["RAG"], n_results=5)

# Print the structure of the query result
# These lines print the structure of the query result to see what keys are available in the returned dictionary.
print("\nQuery Result Structure:")
print(query_result.keys())

# Print the details of the query results
print("\nQuery Results:")

# Enumerate through the Results:
'''The enumerate function adds a counter (idx) to the loop. The zip function combines the corresponding elements from the lists of IDs, metadata, documents, and distances from the query result. 
The get method ensures that if the key is not found, an empty list is returned, preventing errors.'''
for idx, (ids, metadatas, documents, distances) in enumerate(zip(
        query_result.get('ids', [[]])[0],   
        query_result.get('metadatas', [[]])[0],
        query_result.get('documents', [[]])[0],
        query_result.get('distances', [[]])[0]
    )):

    # Print Each Result
    # For each result, it prints the ID, Eskdata (a specific field in metadata), the first 100 characters of the document, and the distance (a measure of similarity).
    print(f"\nResult {idx + 1}:")
    print(f"ID: {ids}")
    print(f"Eskdata: {metadatas.get('eskdata', 'Unknown')}")  # Access the 'eskdata' field in metadata
    print(f"Document (first 100 chars): {documents[:100]}...")  # Print the first 100 characters of the document
    print(f"Distance: {distances:.4f}")

    # Check if embeddings exist before accessing them
    # This part checks if embeddings exist and prints the first 5 values of the embedding for the current result.
    embeddings = query_result.get('embeddings', [])
    if embeddings and embeddings[0]:  # Check both levels
        print(f"Embedding (first 5): {embeddings[0][idx][:5]}...")



Query Result Structure:
dict_keys(['ids', 'distances', 'metadatas', 'embeddings', 'documents', 'uris', 'data', 'included'])

Query Results:

Result 1:
ID: 4
Eskdata: What is Retrieval Augmented Generation (RAG)?
Document (first 100 chars): Title: What is Retrieval Augmented Generation (RAG)?
Link: https://www.datacamp.com/blog/what-is-ret...
Distance: 0.1683

Result 2:
ID: 5
Eskdata: Retrieval-Augmented Generation (RAG) from basics to advanced
Document (first 100 chars): Title: Retrieval-Augmented Generation (RAG) from basics to advanced
Link: https://medium.com/@tejpal...
Distance: 0.1860

Result 3:
ID: 10
Eskdata: An Introduction to Vector Databases For Machine Learning: A Hands-On Guide With Examples
Document (first 100 chars): Title: An Introduction to Vector Databases For Machine Learning: A Hands-On Guide With Examples
Link...
Distance: 0.2848

Result 4:
ID: 15
Eskdata: What Machine Learning Role is Right for You?
Document (first 100 chars): Title: What Machine Learning Role is 

In [4]:
'''This line defines a function named return_best_eskdata that takes three parameters: user_input, collection, and n_results (default is 1). 
The function will find and return the best matching data from the collection based on the user input.'''
def return_best_eskdata(user_input, collection, n_results=1):
    # Query the collection for texts similar to user_input and get the top result
    # This line queries the ChromaDB collection to find the top results that are most similar to the user_input. It returns a dictionary containing the query results.
    query_result = collection.query(query_texts=[user_input], n_results=n_results)
    
    # Check if there are no results
    '''These lines check if there are no results by verifying if the 'ids' key is empty or the first element in the 'ids' list is empty.
    If no results are found, it prints a message and returns None.'''
    if not query_result['ids'] or not query_result['ids'][0]:
        print("No data found matching the query.")
        return None, None  # No results found

    # Get the top result's ID, metadata, and document
    # These lines extract the ID, metadata, and document of the top result from the query results.
    top_result_id = query_result['ids'][0][0]
    top_result_metadata = query_result['metadatas'][0][0]
    top_result_document = query_result['documents'][0][0]
    
    # Print the top result in a readable and formatted manner
    print("Top Data Found:")
    print("---------------")
    print(f"Title: {top_result_metadata.get('eskdata', 'Unknown Data')}")
    print("\nBody:")
    print("-----------------")
    print(top_result_document)
    print("\nRecommendation:")
    print("----------------")
    
    # This line returns the name and document of the top result.
    return top_result_metadata.get('eskdata', 'Unknown Data'), top_result_document


In [5]:
# Example function usage
user_input = "Bag of Words"
top_name, top_document = return_best_eskdata(user_input, collection, n_results=1)

# Print the result
if top_name and top_document:
    print("\nBest Match Found:")
    print(f"Title: {top_name}")
    print(f"Document: {top_document[:100]}...")  # Print the first 100 characters of the document

Top Data Found:
---------------
Title: An Introduction to Bag of Words (BoW)

Body:
-----------------
Title: An Introduction to Bag of Words (BoW)
Link: https://medium.com/@vamshiprakash001/an-introduction-to-bag-of-words-bow-c32a65293ccc
Publish Date: Jun 27, 2023
Sprint: Sprint 4
Body: Vamshi Prakash
Jun 27, 2023

Using Natural Language Processing, we make use of the text data available across the internet to generate insights for the business. In order to understand this huge amount of data and make insights from them, we need to make them usable. Natural language processing helps us to do so.

What is a Bag of Words in NLP?
Bag of words is a Natural Language Processing technique of text modelling. In technical terms, we can say that it is a method of feature extraction with text data. This approach is a simple and flexible way of extracting features from documents.

A bag of words is a representation of text that describes the occurrence of words within a document. We just keep tra

In [6]:
# Import the OpenAI library and initialize the client with the API key
from openai import OpenAI

api_key = os.getenv("OPENAI_API_KEY")
client = OpenAI(api_key=api_key)

'''This line defines a function named generate_conversational_response that takes two parameters: user_input and collection. 
The function will generate a conversational response based on the user input and the collection of data.'''
def generate_conversational_response(user_input, collection):
    # Generate a conversational response based on the user input and collection
    
    # Find the most relevant article based on the user input
    '''This line calls the return_best_eskdata function to find the most relevant article based on the user input and the collection. 
    It returns the name and document of the most relevant article.'''
    relevant_name, relevant_document = return_best_eskdata(user_input, collection)
    
    # If no relevant article is found, return a message indicating so
    # These lines check if no relevant article was found. If no article is found, it returns a message indicating so.
    if not relevant_name:
        return "I couldn't find any relevant articles based on your input."

    # Prepare the messages for the conversational AI
    '''These lines prepare a list of messages to be sent to the OpenAI API. 
    The system message sets the context, the user message contains the user input, and the assistant message contains the recommendation based on the relevant article.'''
    messages = [
        {"role": "system", "content": "You are a bot that makes recommendations for each Sprint 1 to 4 for the Data Science bootcamp."},
        {"role": "user", "content": user_input},
        {"role": "assistant", "content": f"This is the recommended article: {relevant_name}. Here is a brief about the article: {relevant_document}"}
    ]
    
    # Generate a response from the OpenAI API
    # This line calls the OpenAI API to generate a response based on the prepared messages. It uses the gpt-3.5-turbo model and limits the response to 200 tokens.
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=messages,
        max_tokens=200
    )
    
    # Return the generated response
    return response.choices[0].message.content


In [7]:
# Example usage
user_input = "I need to learn Data Visualization"
print(generate_conversational_response(user_input, collection))

Top Data Found:
---------------
Title: Top 50 Matplotlib Visualizations – The Master Plots (with full Python code)


Body:
-----------------
Title: Top 50 Matplotlib Visualizations – The Master Plots (with full Python code)

Link: https://www.machinelearningplus.com/plots/top-50-matplotlib-visualizations-the-master-plots-python/
Publish Date: nan
Sprint: Sprint 1
Body: Correlation

Scatter plot
Scatterplot is a classic and fundamental plot used to study the relationship between two variables. If you have multiple groups in your data you may want to visualise each group in a different color. In matplotlib, you can conveniently do this using plt.scatterplot().

python
Copy code
import matplotlib.pyplot as plt
import seaborn as sns

df = sns.load_dataset('iris')
sns.scatterplot(x='sepal_length', y='sepal_width', hue='species', data=df)
plt.show()
Bubble plot with Encircling
Sometimes you want to show a group of points within a boundary to emphasize their importance. In this example, you g